## This notebook shows how to generate Diverse Counterfactual Explanations and how to use some of the options available to generate explanations

In [1]:
## import libraries

## python libraries
import numpy as np
import pandas as pd
import os

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras

# DiCE libraries
import dice_ml
from dice_ml import dice # dice interface
from dice_ml import data # data interface
from dice_ml import model # model interface
from dice_ml.utils import helpers # helper functions

### Loading dataset

We use popular 'adult' income dataset from UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/adult). For demonstration purposes, we transform the data as detailed in **detail in dice_ml.utils.helpers** module. 

In [2]:
dataset = helpers.load_adult_income_dataset()

In [3]:
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,39,Government,Bachelors,Single,White-Collar,White,Male,40,0
1,50,Self-Employed,Bachelors,Married,White-Collar,White,Male,13,0
2,38,Private,HS-grad,Divorced,Blue-Collar,White,Male,40,0
3,53,Private,School,Married,Blue-Collar,Other,Male,40,0
4,28,Private,Bachelors,Married,Professional,Other,Female,40,0


In [4]:
d = data.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')

### Training ML model

Below, we build an Artificial Neural Network based on Keras Tensorflow framework.

In [5]:
# seeding random numbers for reproducability
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [6]:
sess = tf.InteractiveSession()

train, _ = d.split_data(d.normalize_data(d.one_hot_encoded_data))
X_train = train.loc[:, train.columns != 'income']
y_train = train.loc[:, train.columns == 'income']

ann_model = keras.Sequential()
ann_model.add(keras.layers.Dense(20, input_shape=(X_train.shape[1],), kernel_regularizer=keras.regularizers.l2(0.01), activation=tf.nn.relu))
ann_model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

ann_model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.01), metrics=['accuracy'])
ann_model.fit(X_train, y_train, validation_split=0.20, epochs=50, verbose=0)


In [7]:
m = model.Model(model=ann_model) 

### Generate diverse counterfactuals

In [8]:
# initiate DiCE
exp = dice.Dice(d, m)

In [9]:
# query instance in the form of a dictionary; keys: feature name, values: feature value
query_instance = {'age':22, 
                  'workclass':'Private', 
                  'education':'HS-grad', 
                  'marital_status':'Single', 
                  'occupation':'Service',
                  'race': 'White', 
                  'gender':'Female', 
                  'hours_per_week': 45}

In [10]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

Diverse Counterfactuals found! total time taken: 00 min 10 sec


In [11]:
# visualize the resutls
dice_exp.visualize_as_dataframe()

Query instance:


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.016217



Diverse Counterfactual set:


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,90.0,Government,HS-grad,Married,Service,White,Female,99.0,0.715
1,90.0,Private,Prof-school,Single,White-Collar,White,Female,99.0,0.716
2,58.0,Private,Masters,Married,Professional,Other,Female,99.0,0.844
3,90.0,Private,Bachelors,Married,Service,White,Male,55.0,0.840


As we see from above resutls, ... **Todo: some comments about the results**

### Feature weights

Users can add feature weights.. feasibility constraints **# Todo: add description**

Below, we use the inverse of Median Absolute Deviation as feature weights

In [12]:
# get MAD
mads = d.get_mads_from_training_data(normalized=True)

# design feature weights
feature_weights = {}
for feature in mads:
    feature_weights[feature] = round(1/mads[feature], 2)
print(feature_weights)

{'age': 7.3, 'hours_per_week': 24.5}


In [13]:
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite", 
                                        feature_weights=feature_weights)

Diverse Counterfactuals found! total time taken: 00 min 10 sec


In [14]:
dice_exp.visualize_as_dataframe()

Query instance:


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.016217



Diverse Counterfactual set:


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,60.0,Private,Prof-school,Married,White-Collar,White,Female,41.0,0.844
1,37.0,Private,Doctorate,Married,White-Collar,White,Male,77.0,0.844
2,90.0,Private,Prof-school,Married,Service,White,Female,42.0,0.844
3,30.0,Private,Prof-school,Married,Professional,White,Male,73.0,0.844


As we see from above illustration,...

### Proximity and Diversity weights

These two weights balances the three parts of loss function... **# Todo: add description**

Let us try to change the default values of these weights to see if we get any changes..

In [15]:
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite", 
                                        feature_weights=feature_weights, 
                                        proximity_weight=1.5, diversity_weight=1.0) 

Diverse Counterfactuals found! total time taken: 00 min 09 sec


In [16]:
dice_exp.visualize_as_dataframe()

Query instance:


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.016217



Diverse Counterfactual set:


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,50.0,Private,Doctorate,Married,White-Collar,White,Male,37.0,0.841
1,81.0,Private,Prof-school,Married,Service,White,Female,63.0,0.844
2,37.0,Private,Prof-school,Married,Professional,White,Female,3.0,0.609
3,23.0,Private,Prof-school,Married,White-Collar,White,Female,92.0,0.844


As we see from above illustration,...

### Features to vary

Let us now vary only those features for which perturbing makes sense.. **# Todo: add description**

In [17]:
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite", 
                                        feature_weights=feature_weights, 
                                        features_to_vary=['age','workclass','education','occupation','hours_per_week'])

Diverse Counterfactuals found! total time taken: 00 min 13 sec


In [18]:
dice_exp.visualize_as_dataframe()

Query instance:


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.016217



Diverse Counterfactual set:


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,90.0,Government,Prof-school,Single,White-Collar,White,Female,99.0,0.726
1,90.0,Self-Employed,Doctorate,Single,White-Collar,White,Female,99.0,0.670
2,90.0,Government,Doctorate,Single,Professional,White,Female,99.0,0.673
3,90.0,Private,Prof-school,Single,Professional,White,Female,99.0,0.709


As we see from above illustration... without changing race or gender, this ML model is not able to provide valid diverse counterfactuals...